## Motivation {.unnumbered}
Single cell omic analysis can be done on both R or Python. There currently exists a few packages to format, process & analyse scRNAseq data, namely Seurat (V5) [[Hao et al.; 2023]](https://www.nature.com/articles/s41587-023-01767-y), Scanpy [[Wolf et al.; 2018]](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-017-1382-0), SingleCellExperiment [[Amezquita et al.; 2019]](https://www.nature.com/articles/s41592-019-0654-x), Scran [[Lun et al.; 2016]](https://f1000research.com/articles/5-2122/v2). In this tutorial we will focus on using Seurat (V5) due to its comprehensive functionality to handle multimodal datasets and interoperability with other data formats. For larger datasets (>100k cells), we recommend using Scanpy to speed up the processing time.

In [1]:
## set up environment
suppressMessages({
library(scUnify)
setwd("/nemo/lab/caladod/working/Matthew/project/matthew/MH_GSE247917")})

Warning message:
“replacing previous import ‘cowplot::get_legend’ by ‘ggpubr::get_legend’ when loading ‘scUnify’”
Warning message:
“replacing previous import ‘cowplot::align_plots’ by ‘patchwork::align_plots’ when loading ‘scUnify’”
Warning message:
“replacing previous import ‘biomaRt::select’ by ‘rstatix::select’ when loading ‘scUnify’”
Warning message:
“replacing previous import ‘scales::viridis_pal’ by ‘viridis::viridis_pal’ when loading ‘scUnify’”


## Import CellRanger Outputs
Now we will import the outputs from cellranger-multi as a Seurat object. We will first need to specify a cellranger-multi output directory and a sample name for each sequencing run.

In [4]:
## store a list of 10x output directories as a vector & define sample names
dir <- "/nemo/lab/caladod/scratch/hungm/matthew/MH_GSE247917/cellranger/"
samples = list.files(dir)
dir.list <- paste0(dir, samples, "/outs/per_sample_outs/", samples, "/count/sample_filtered_feature_bc_matrix/")
dir.list

[1] "/nemo/lab/caladod/scratch/hungm/matthew/MH_GSE247917/cellranger/CV10/outs/per_sample_outs/CV10/count/sample_filtered_feature_bc_matrix/"
[2] "/nemo/lab/caladod/scratch/hungm/matthew/MH_GSE247917/cellranger/CV12/outs/per_sample_outs/CV12/count/sample_filtered_feature_bc_matrix/"

The "create_seurat_object" function is a wrapper to :

1. Build a list of seurat object from a list of specified cellranger-multi output directories
2. Store gene expression counts in the "RNA" assay of each object. Cells with < 200 nFeatures_RNA and genes expressed in < 3 cells will be pre-filtered.
3. Extract cell hashing counts based on the HTO string (hto_str) given and store in the "HTO" assay of each object. All HTO antibody names should have common prefix/suffix.
4. Store CITEseq antibody counts in the "ADT" assay of each object and performs DSB normalization (if adt_normalized = T)
5. Sample names are specified in the "samples" column in each Seurat object's metadata. 

In [23]:
## build seurat object with RNA, HTO & ADT assays, specifying HTO strings and normalizing ADT counts
obj.list <- create_seurat_object(dir = dir.list, samples = samples, hto_str = "^CV", adt_normalize = T)

CV10 --- Loading Sample 1

Step 1 : Adding RNA counts

10X data contains more than one type and is being returned as a list containing matrices of each type.

Step 2 : Check for Modalities

Step 3 : Adding HTO counts

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Step 4 : Adding ADT counts

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Warning message in dsb::DSBNormalizeProtein(cell_protein_matrix = adt.cell, empty_drop_matrix = adt.raw, :
“`use.isotype.control` = FALSE is not recommended if setting `denoise.counts` = TRUE 
when isotype controls are available.
 If data include isotype controls set `denoise.counts` = TRUE `use.isotype.control` = TRUE

[1] "potential isotype controls detected: "
[1] "IsoIgG2A" "IsoIgG1" 
[1] "correcting ambient protein background noise"
[1] "some proteins with low background variance detected check raw and normalized distributions.  protein stats can be returned with return.stats = TRUE"
[1] "EpCAM"
[1] "fitting models to each cell for dsb technical component and removing cell to cell technical noise"


Step 5 : Creating Seurat Object

CV12 --- Loading Sample 2

Step 1 : Adding RNA counts

10X data contains more than one type and is being returned as a list containing matrices of each type.

Step 2 : Check for Modalities

Step 3 : Adding HTO counts

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Step 4 : Adding ADT counts

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Warning message in dsb::DSBNormalizeProtein(cell_protein_matrix = adt.cell, empty_drop_matrix = adt.raw, :
“`use.isotype.control` = FALSE is not recommended if setting `denoise.counts` = TRUE 
when isotype controls are available.
 If data include isotype controls set `denoise.counts` = 

[1] "potential isotype controls detected: "
[1] "IsoIgG2A" "IsoIgG1" 
[1] "correcting ambient protein background noise"
[1] "some proteins with low background variance detected check raw and normalized distributions.  protein stats can be returned with return.stats = TRUE"
 [1] "CD186"   "CD1a"    "CD39"    "ICOS.1"  "CD11b"   "CD1c"    "CD134"  
 [8] "CD27.1"  "CD152"   "CD197"   "CD69.1"  "CD103"   "CD185"   "EpCAM"  
[15] "CD8"     "CD123"   "CD117"   "HLA-ABC" "CD19.1"  "CD86.1" 
[1] "fitting models to each cell for dsb technical component and removing cell to cell technical noise"


Step 5 : Creating Seurat Object



Finally, a quick check if the Seurat objects are set up properly.

In [25]:
## View seurat object list
obj.list

$CV10
An object of class Seurat 
23862 features across 19484 samples within 3 assays 
Active assay: RNA (23797 features, 0 variable features)
 1 layer present: counts
 2 other assays present: HTO, ADT

$CV12
An object of class Seurat 
16451 features across 17366 samples within 3 assays 
Active assay: RNA (16386 features, 0 variable features)
 1 layer present: counts
 2 other assays present: HTO, ADT


In [26]:
## View metadata the first seurat object
head(obj.list[[1]])

,orig.ident,nCount_RNA,nFeature_RNA,nCount_HTO,nFeature_HTO,nCount_ADT,nFeature_ADT,samples
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<chr>
CV10_AAACCTGAGAAGCCCA-1,SeuratProject,2075,1226,91,3,1336,58,CV10
CV10_AAACCTGAGAGGGATA-1,SeuratProject,4113,2208,257,3,1008,57,CV10
CV10_AAACCTGAGAGTGACC-1,SeuratProject,1218,786,88,2,829,57,CV10
CV10_AAACCTGAGATATGGT-1,SeuratProject,6011,2869,599,4,3661,57,CV10
CV10_AAACCTGAGCCAGTAG-1,SeuratProject,1322,777,70,3,1216,58,CV10
CV10_AAACCTGAGCGATATA-1,SeuratProject,1743,1134,84,4,700,57,CV10
CV10_AAACCTGAGGAATCGC-1,SeuratProject,2892,1617,147,3,1479,58,CV10
CV10_AAACCTGAGTCAAGCG-1,SeuratProject,1258,808,108,2,1063,55,CV10
CV10_AAACCTGAGTGGAGTC-1,SeuratProject,1918,1312,62,4,20921,58,CV10


In [28]:
## View assays in first seurat object
for(a in names(obj.list[[1]]@assays)){
    print(obj.list[[1]][[a]])}

Assay (v5) data with 23797 features for 19484 cells
First 10 features:
 AL627309.1, AL627309.5, AP006222.2, LINC01409, FAM87B, LINC01128,
LINC00115, FAM41C, AL645608.6, SAMD11 
Layers:
 counts 
Assay (v5) data with 4 features for 19484 cells
First 4 features:
 CV-011-d120-booster, CV-001-d28-booster, CV-001-d7-booster,
CV-001-d0-booster 
Layers:
 counts 
Assay (v5) data with 61 features for 19484 cells
First 10 features:
 CD138, CD186, CD38.1, CD1a, CD26, CD45, CD127, CD28.1, CD2.1, CD184 
Layers:
 counts, data 


## Session Info {.unnumbered}

In [ ]:
## save output of the this session
qsave(obj.list, file = "seurat/1_processing/1.2_GSE247917_raw.qs")

In [30]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.7 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /nemo/lab/caladod/working/Matthew/.conda/envs/seurat5/lib/libopenblasp-r0.3.23.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/London
tzcode source: system (glibc)

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] scUnify_0.0.0.9000          ComplexHeatmap_2.16.0      
 [3] DoubletFinder_2.0.4         scDblFinder_1.14.0         
 [5] celda_1.16.1                Matrix_1.6-1 